In [13]:
import os
import pandas as pd
import numpy as np
import natsort


In [14]:
'''
    Delete duplicated and return a clear insurance df.
'''
def pre_process_insurance(file_name):
    file = pd.read_excel(file_name, sheet_name='for insurance 202101')
    # First, need to delete duplicated lines, and get a clean df
    df = pd.DataFrame(file)
    df_duplicated = df[df[['发票合同号码','启运日期']].duplicated()]
    
#     print("Found duplicated error.")
#     print(df_duplicated)
#     df = df[~df[['发票合同号码','启运日期']].duplicated()]
    # return the full df
    return df

In [15]:
'''
    Do not delete duplicated index, and return a duplicated index.
'''
def find_dup(file_name):
    file = pd.read_excel(file_name, sheet_name='for insurance 202101')
    # First, need to delete duplicated lines, and get a clean df
    df = pd.DataFrame(file)
    df_duplicated = df[df[['发票合同号码','启运日期']].duplicated()]
    
    print("Found duplicated error.")
    print(df_duplicated)
    # df = df[~df[['发票合同号码','启运日期']].duplicated()]
    # return dp_index
    dp_index = df_duplicated['序号'].tolist()
    return dp_index

In [17]:
def check_by_insurance(insurance_df, data_df, dp_index):
    insur_array = np.array(insurance_df)
    data_array = np.array(data_df)
    result_df = pd.DataFrame(columns=['序号','发票合同号码','货物名称','数量','航班号','启运日期','启运地','目的地','保险金额','运单号','Document No.','Match_Status'])
    for i in range(len(insur_array)):
        count = insur_array[i,0]
        order_id = insur_array[i,1]
        flight = insur_array[i,4]
        date = insur_array[i,5]
        
        # if this index is duplicated, we do not process this.
        if count in dp_index:
            new_data = {
                    '序号' :insur_array[i,0],
                    '货物名称' : insur_array[i,2],
                    '数量' : insur_array[i,3],
                    '启运地' : insur_array[i,6],
                    '目的地' : insur_array[i,7],
                    '保险金额' : insur_array[i,8],
                    '运单号' : insur_array[i,9],
                    '发票合同号码' : order_id,
                    '航班号' : flight,
                    '启运日期' : date,
                    'Match_Status' : 'duplicated'
                }
            result_df = result_df.append(new_data,ignore_index=True)
            continue
            
        
        locs = np.argwhere(data_array==order_id)
        
        if(len(locs)==0):
            # cannot find matching order ID
            new_data = {
                    '序号' :insur_array[i,0],
                    '货物名称' : insur_array[i,2],
                    '数量' : insur_array[i,3],
                    '启运地' : insur_array[i,6],
                    '目的地' : insur_array[i,7],
                    '保险金额' : insur_array[i,8],
                    '运单号' : insur_array[i,9],
                    '发票合同号码' : order_id,
                    '航班号' : flight,
                    '启运日期' : date,
                    'Match_Status' : 'Empty'
                }
            result_df = result_df.append(new_data,ignore_index=True)
            continue
        
        flag = 0
        for loc in locs:
            index = loc[0]
            
            if(data_array[index,1]==date):
                new_data = {
                    '序号' :insur_array[i,0],
                    '货物名称' : insur_array[i,2],
                    '数量' : insur_array[i,3],
                    '启运地' : insur_array[i,6],
                    '目的地' : insur_array[i,7],
                    '保险金额' : insur_array[i,8],
                    '运单号' : insur_array[i,9],
                    '发票合同号码' : order_id,
                    '航班号' : flight,
                    '启运日期' : date,
                    '保险金额' : data_array[index,3],
                    'Document No.' : data_array[index,2],
                    'Match_Status' : 'True'
                }
                result_df = result_df.append(new_data,ignore_index=True)
                flag = 1
                break
            else:
                continue
        if(flag==0):
            # do not match any date
            new_data = {
                    '序号' :insur_array[i,0],
                    '货物名称' : insur_array[i,2],
                    '数量' : insur_array[i,3],
                    '启运地' : insur_array[i,6],
                    '目的地' : insur_array[i,7],
                    '保险金额' : insur_array[i,8],
                    '运单号' : insur_array[i,9],
                    '发票合同号码' : order_id,
                    '航班号' : flight,
                    '启运日期' : date,
                    'Match_Status' : 'DateError'
                }
            result_df = result_df.append(new_data,ignore_index=True)
    return result_df
        

In [14]:
'''
    test for data sheet
'''
file = pd.read_excel('../test/For Insurance 202101.xlsx', sheet_name='data')
data_df = pd.DataFrame(file)
data_array = np.array(data_df)
locs = np.argwhere(data_array=='SOM143907')
for loc in locs:
    index = loc[0]
    print(data_array[index,2])
    print(data_array[index,1])

IS1429810
2021-01-05 00:00:00


In [19]:
'''
    main
'''
file = pd.read_excel('../test/For Insurance 202101.xlsx', sheet_name='data')
data_df = pd.DataFrame(file)
dp_index = find_dup('../test/For Insurance 202101.xlsx')
df = pre_process_insurance('../test/For Insurance 202101.xlsx')
result_df = check_by_insurance(df,data_df, dp_index)
result_df

Found duplicated error.
      序号     发票合同号码  货物名称  数量  航班号       启运日期 启运地 目的地  保险金额           运单号  \
318  599  SOM142487  手表首饰  16  NaN 2021-01-19  上海  上海   NaN  CAE689105859   
319  600  SOM142487  手表首饰  10  NaN 2021-01-19  上海  上海   NaN  CAE689105859   

     Document No.  
318           NaN  
319           NaN  
Found duplicated error.
      序号     发票合同号码  货物名称  数量  航班号       启运日期 启运地 目的地  保险金额           运单号  \
318  599  SOM142487  手表首饰  16  NaN 2021-01-19  上海  上海   NaN  CAE689105859   
319  600  SOM142487  手表首饰  10  NaN 2021-01-19  上海  上海   NaN  CAE689105859   

     Document No.  
318           NaN  
319           NaN  


,序号,发票合同号码,货物名称,数量,航班号,启运日期,启运地,目的地,保险金额,运单号,Document No.,Match_Status
0,317,SOM000384,手表首饰,350,CA1558,2021-01-12,上海,北京,335862.50,SC215161,PSI036267,True
1,318,SOM000385,手表首饰,20,CA1558,2021-01-12,上海,北京,18655.30,SC215161,PSI036266,True
2,310,SOM000386,手表首饰,266,HU7616,2021-01-08,上海,北京,132411.28,SC215159,PSI036216,True
3,313,SOM000387,手表首饰,514,HU7616,2021-01-08,上海,北京,373039.07,SC215160,PSI036214,True
4,314,SOM000388,手表首饰,35,HU7616,2021-01-08,上海,北京,34956.81,SC215160,PSI036215,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1424,1428,TO139520,手表首饰,1,CZ8888,2021-01-28,上海,北京,236.55,CAE689108016,TS127510,True
1425,1429,TO139521,手表首饰,1,NaN,2021-01-28,上海,苏州,236.55,CAE689107885,TS127514,True
1426,1430,TO139522,手表首饰,1,CA1837,2021-01-28,上海,广州,236.55,CAE689108047,TS127569,True
1427,1431,TO139523,手表首饰,1,NaN,2021-01-28,上海,上海,236.55,CAE689107775,TS127521,True


In [20]:
result_df.to_excel('../test/8-12/result_for_insurance.xlsx')